In [24]:
%pip install xgboost
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn

Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Active code page: 1252
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
%matplotlib inline
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error


In [26]:
data = pd.read_csv('valve_final.csv')

In [27]:
data.head(8)

,Date,Time,Player_count
0,24/10/2023,14:05,609952
1,24/10/2023,14:10,608506
2,24/10/2023,14:15,606577
3,24/10/2023,14:20,604879
4,24/10/2023,14:25,603868
5,24/10/2023,14:30,603070
6,24/10/2023,14:35,602284
7,24/10/2023,14:40,597245


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4202 entries, 0 to 4201
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          4202 non-null   object
 1   Time          4202 non-null   object
 2   Player_count  4202 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 98.6+ KB


In [29]:
data = data.drop(data[(data["Player_count"]== 0)].index)

In [30]:
data["New_date"] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3877 entries, 0 to 4201
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3877 non-null   object        
 1   Time          3877 non-null   object        
 2   Player_count  3877 non-null   int64         
 3   New_date      3877 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 151.4+ KB


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3877 entries, 0 to 4201
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3877 non-null   object        
 1   Time          3877 non-null   object        
 2   Player_count  3877 non-null   int64         
 3   New_date      3877 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 151.4+ KB


### Feature engineering (transform data to be used for the machine learning algo) 

In [33]:

data["New_time"] = pd.to_datetime(data['Time'], format='%H:%M')

In [34]:
data["year"] = data["New_date"].dt.year
data["month"] = data["New_date"].dt.month
data["day"] = data["New_date"].dt.day
data["hour"] = data["New_time"].dt.hour
data["min"] = data["New_time"].dt.minute

In [35]:
data.drop(columns=["Date","Time","New_date","New_time"], inplace=True)

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3877 entries, 0 to 4201
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Player_count  3877 non-null   int64
 1   year          3877 non-null   int32
 2   month         3877 non-null   int32
 3   day           3877 non-null   int32
 4   hour          3877 non-null   int32
 5   min           3877 non-null   int32
dtypes: int32(5), int64(1)
memory usage: 136.3 KB


In [37]:
data.head()

,Player_count,year,month,day,hour,min
0,609952,2023,10,24,14,5
1,608506,2023,10,24,14,10
2,606577,2023,10,24,14,15
3,604879,2023,10,24,14,20
4,603868,2023,10,24,14,25


### Preapering for training 

### Models and testing

#### xgboost

In [38]:
tss = TimeSeriesSplit(n_splits=5, test_size=11, gap=11)
df = data.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    FEATURES = ['year', 'month', 'day', 'hour', 'min']
    TARGET = 'Player_count'

    X_train = train[FEATURES].values
    y_train = train[TARGET]

    X_test = test[FEATURES].values
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:40:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:906825.61352	validation_1-rmse:722343.81886
[100]	validation_0-rmse:357933.53846	validation_1-rmse:308534.90451
[200]	validation_0-rmse:175088.55595	validation_1-rmse:140072.39271
[300]	validation_0-rmse:121766.04453	validation_1-rmse:88728.04475


[400]	validation_0-rmse:107853.70039	validation_1-rmse:69514.13018
[500]	validation_0-rmse:103341.70860	validation_1-rmse:59643.22991
[600]	validation_0-rmse:100771.76019	validation_1-rmse:55089.03667
[700]	validation_0-rmse:99351.50649	validation_1-rmse:50339.61213
[800]	validation_0-rmse:98162.59752	validation_1-rmse:48341.34592
[900]	validation_0-rmse:97233.39150	validation_1-rmse:46260.61635
[999]	validation_0-rmse:96489.09874	validation_1-rmse:45416.11229
[0]	validation_0-rmse:906272.98977	validation_1-rmse:754476.86204
[100]	validation_0-rmse:357704.20891	validation_1-rmse:302537.60225
[200]	validation_0-rmse:175001.17004	validation_1-rmse:130285.21351
[300]	validation_0-rmse:121614.07453	validation_1-rmse:79446.44734
[400]	validation_0-rmse:107615.72009	validation_1-rmse:59327.12640


C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:40:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[500]	validation_0-rmse:103096.10853	validation_1-rmse:49274.05867
[600]	validation_0-rmse:100602.81366	validation_1-rmse:42395.70270
[700]	validation_0-rmse:99199.34393	validation_1-rmse:37107.33679
[800]	validation_0-rmse:98027.53356	validation_1-rmse:35143.76814
[900]	validation_0-rmse:97080.71174	validation_1-rmse:33083.23062
[999]	validation_0-rmse:96328.59871	validation_1-rmse:32497.58400
[0]	validation_0-rmse:905796.79614	validation_1-rmse:804815.39895
[100]	validation_0-rmse:357489.14242	validation_1-rmse:306011.21866
[200]	validation_0-rmse:174855.61113	validation_1-rmse:131667.63808
[300]	validation_0-rmse:121523.17080	validation_1-rmse:79656.26561
[400]	validation_0-rmse:107492.02490	validation_1-rmse:59749.96586


...

[500]	validation_0-rmse:102928.62832	validation_1-rmse:49788.23335
[600]	validation_0-rmse:100406.01425	validation_1-rmse:43655.66395
[700]	validation_0-rmse:98962.99420	validation_1-rmse:37764.78113
[800]	validation_0-rmse:97803.44526	validation_1-rmse:36130.05125
[900]	validation_0-rmse:96808.59291	validation_1-rmse:33786.26809
[999]	validation_0-rmse:96057.28410	validation_1-rmse:32255.10760
[0]	validation_0-rmse:905397.46681	validation_1-rmse:895404.98144
[100]	validation_0-rmse:357300.79525	validation_1-rmse:336834.43925
[200]	validation_0-rmse:175059.94045	validation_1-rmse:154333.61052
[300]	validation_0-rmse:121553.09979	validation_1-rmse:100909.06169
[400]	validation_0-rmse:107506.95218	validation_1-rmse:80889.61673


C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:40:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[500]	validation_0-rmse:102974.70803	validation_1-rmse:68808.71281
[600]	validation_0-rmse:100416.25090	validation_1-rmse:59173.79280
[700]	validation_0-rmse:99006.77270	validation_1-rmse:53126.29924
[800]	validation_0-rmse:97907.77427	validation_1-rmse:51175.31095
[900]	validation_0-rmse:96921.62065	validation_1-rmse:49058.75984
[999]	validation_0-rmse:96190.48409	validation_1-rmse:47677.97247
[0]	validation_0-rmse:905124.83833	validation_1-rmse:1018547.30064
[100]	validation_0-rmse:357132.50983	validation_1-rmse:396087.80473
[200]	validation_0-rmse:175012.35104	validation_1-rmse:199207.17166
[300]	validation_0-rmse:121401.67267	validation_1-rmse:133769.48518
[400]	validation_0-rmse:107313.68117	validation_1-rmse:109332.94307


C:\Users\Tommy\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:40:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[500]	validation_0-rmse:102720.86304	validation_1-rmse:92899.36649
[600]	validation_0-rmse:100265.49165	validation_1-rmse:81513.58790
[700]	validation_0-rmse:98867.97706	validation_1-rmse:74546.52621
[800]	validation_0-rmse:97783.43101	validation_1-rmse:72162.22532
[900]	validation_0-rmse:96826.03745	validation_1-rmse:69992.78849
[999]	validation_0-rmse:96098.14258	validation_1-rmse:68580.15149


In [39]:
print(f'Score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

Score across folds 45261.6469
Fold scores:[45365.81726514566, 32476.8851848021, 32233.450132388014, 47655.75934755617, 68576.32240536474]


Saving model to pkl file for later use


In [40]:
reg.save_model("reg_model_valve.json")
